<a href="https://colab.research.google.com/github/GreaterthanGreat/Dish-Detect-App/blob/main/Skin_Disease_Ml_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install Dependencies
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
from google.colab import files
Train_Dataset = files.upload()

Saving train.zip to train (1).zip


In [3]:
!unzip train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/benign/melanoma_4770.jpg  
  inflating: __MACOSX/train/benign/._melanoma_4770.jpg  
  inflating: train/benign/melanoma_2301.jpg  
  inflating: __MACOSX/train/benign/._melanoma_2301.jpg  
  inflating: train/benign/melanoma_1608.jpg  
  inflating: __MACOSX/train/benign/._melanoma_1608.jpg  
  inflating: train/benign/melanoma_92.jpg  
  inflating: __MACOSX/train/benign/._melanoma_92.jpg  
  inflating: train/benign/melanoma_191.jpg  
  inflating: __MACOSX/train/benign/._melanoma_191.jpg  
  inflating: train/benign/melanoma_2467.jpg  
  inflating: __MACOSX/train/benign/._melanoma_2467.jpg  
  inflating: train/benign/melanoma_3779.jpg  
  inflating: __MACOSX/train/benign/._melanoma_3779.jpg  
  inflating: train/benign/melanoma_4016.jpg  
  inflating: __MACOSX/train/benign/._melanoma_4016.jpg  
  inflating: train/benign/melanoma_2473.jpg  
  inflating: __MACOSX/train/benign/._melanoma_2473.jpg  
  inflating: train/benign/me

In [4]:
import os
from google.colab import files
Test_Dataset = files.upload()

Saving test.zip to test (1).zip


In [5]:
!unzip test.zip

Archive:  test.zip
replace __MACOSX/._test? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/._test         
  inflating: __MACOSX/test/._malignant  
  inflating: __MACOSX/test/._benign  
  inflating: test/malignant/melanoma_10207.jpg  
  inflating: __MACOSX/test/malignant/._melanoma_10207.jpg  
  inflating: test/malignant/melanoma_10561.jpg  
  inflating: __MACOSX/test/malignant/._melanoma_10561.jpg  
  inflating: test/malignant/melanoma_10575.jpg  
  inflating: __MACOSX/test/malignant/._melanoma_10575.jpg  
  inflating: test/malignant/melanoma_10213.jpg  
  inflating: __MACOSX/test/malignant/._melanoma_10213.jpg  
  inflating: test/malignant/melanoma_10549.jpg  
  inflating: __MACOSX/test/malignant/._melanoma_10549.jpg  
  inflating: test/malignant/melanoma_10373.jpg  
  inflating: __MACOSX/test/malignant/._melanoma_10373.jpg  
  inflating: test/malignant/melanoma_10415.jpg  
  inflating: __MACOSX/test/malignant/._melanoma_10415.jpg  
  inflating: test/malignant/melanoma

In [6]:
os.chdir("/content")
os.listdir()

['.config',
 'train.zip',
 'test.zip',
 'test',
 '__MACOSX',
 'train',
 'test (1).zip',
 'train (1).zip',
 'sample_data']

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Traingen = ImageDataGenerator(rescale=1./255)
Testgen = ImageDataGenerator(rescale=1./255)

train_generator = Traingen.flow_from_directory(
    '/content/train',
    target_size=(256, 256),
    batch_size=90,
    class_mode='categorical',
    shuffle=True
)

test_generator = Testgen.flow_from_directory(
    '/content/test',
    target_size=(256, 256),
    batch_size= 90,
    shuffle=True,
    class_mode='categorical'
)

Found 9605 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.regularizers import l2

base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(256,256,3))

finetune_afterwards = 270

for layer in base_model.layers[:finetune_afterwards]:
    layer.trainable = False

GAP_2D = GlobalAveragePooling2D()
Dense1 = Dense(800, activation='relu', kernel_regularizer=l2(0.001))
Dropout1 = Dropout(0.2)
Dense2 = Dense(300, activation='relu', kernel_regularizer=l2(0.001))
Dropout2 = Dropout(0.2)
prediction = Dense(2, activation='softmax')

model = Sequential([
    base_model,
    GAP_2D,
    Dense1,
    Dropout1,
    Dense2,
    Dropout2,
    prediction
])


In [9]:
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import EarlyStopping
import pickle

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=2,  restore_best_weights=True)

STEP_SIZE_TRAIN = 105
STEP_SIZE_TEST = 10


run = model.fit(train_generator,
                steps_per_epoch= STEP_SIZE_TRAIN,
                validation_data= test_generator,
                validation_steps= STEP_SIZE_TEST,
                epochs=5
)

pickle.dump(model, open('model.pkl', 'wb'))

Epoch 1/5
105/105 [==============================] - 1718s 16s/step - loss: 1.2945 - accuracy: 0.8900 - val_loss: 0.8509 - val_accuracy: 0.8933
Epoch 2/5
105/105 [==============================] - 1710s 16s/step - loss: 0.5616 - accuracy: 0.9420 - val_loss: 0.5103 - val_accuracy: 0.9144
Epoch 3/5
105/105 [==============================] - 1720s 16s/step - loss: 0.2990 - accuracy: 0.9625 - val_loss: 0.3869 - val_accuracy: 0.9144
Epoch 4/5
105/105 [==============================] - 1714s 16s/step - loss: 0.1792 - accuracy: 0.9736 - val_loss: 0.4670 - val_accuracy: 0.9078
Epoch 5/5
105/105 [==============================] - 1722s 16s/step - loss: 0.1093 - accuracy: 0.9838 - val_loss: 0.4529 - val_accuracy: 0.9067


In [10]:
scores = model.evaluate(test_generator, steps=STEP_SIZE_TEST)
print(scores)

10/10 [==============================] - 130s 13s/step - loss: 0.4718 - accuracy: 0.9056
[0.47179993987083435, 0.9055555462837219]
